# 라이브러리 및 세팅

In [8]:
import requests
import json
import pandas as pd
from tqdm.auto import tqdm

In [3]:
data_path = 'data/crawling_data'
OC = "khs980714"

url = "https://www.law.go.kr/"

# 판례 데이터

## 판례 목록

In [4]:
def page_precsearch(page):
  precsearch_url = "/DRF/lawSearch.do?"

  params = {
    "OC": OC,
    "target": "prec",
    "type": "json",
    "display" : 100,
    "JO": "민법",
    "page": page
  }

  response = requests.get(url + precsearch_url, params=params)
  data = response.json()

  df = pd.DataFrame(data["PrecSearch"]['prec']).iloc[:, 1:]
  return df

In [10]:
precsearch_df = pd.DataFrame()

for i in tqdm(range(0, 100)):
  new_df = page_precsearch(i)
  precsearch_df = pd.concat([precsearch_df, new_df])

precsearch_df.shape

  0%|          | 0/100 [00:00<?, ?it/s]

(10000, 12)

In [17]:
precsearch_df.head()

,사건번호,데이터출처명,사건종류코드,사건종류명,선고,선고일자,판례일련번호,판결유형,법원종류코드,법원명,판례상세링크,사건명
0,2024다302217,대법원,400101,민사,선고,2025.03.27,605337,판결,NaN,대법원,/DRF/lawService.do?OC=khs980714&target=prec&ID...,임대료등청구
1,2023다283401,대법원,400101,민사,선고,2025.03.27,605333,판결,NaN,대법원,/DRF/lawService.do?OC=khs980714&target=prec&ID...,손해배상(기)
2,2021다245528,대법원,400101,민사,선고,2025.03.27,605281,판결,NaN,대법원,/DRF/lawService.do?OC=khs980714&target=prec&ID...,근로자지위확인등·근로자지위확인등
3,2024스866,대법원,400103,가사,자,2025.03.24,605331,결정,NaN,대법원,/DRF/lawService.do?OC=khs980714&target=prec&ID...,상속재산분할·기여분·상속재산분할
4,2024다315046,대법원,400101,민사,선고,2025.03.13,605077,판결,NaN,대법원,/DRF/lawService.do?OC=khs980714&target=prec&ID...,차임증액


In [11]:
precsearch_df.to_csv(f"{data_path}/precsearch_df.csv", index=False)

## 판례 상세

In [58]:
precsearch_df = pd.read_csv(f"{data_path}/precsearch_df.csv")

In [57]:
def page_precservice(ID):
  precservice_url = "/DRF/lawService.do?"

  params = {
    "OC": OC,
    "target": "prec",
    "type": "json",
    "ID": ID
  }

  response = requests.get(url + precservice_url, params=params)
  data = response.json()

  return data['PrecService']

In [61]:
for n in tqdm(precsearch_df['판례일련번호']):
  data = page_precservice(n)

  with open(f"{data_path}/{n}.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

  0%|          | 0/10000 [00:00<?, ?it/s]